# AGN in ECDFS location

- author Sylvie Dagoret-Campagne
- creation date 2025-06-07
- last update 2025-06-07


## Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

# import lsst.daf.butler as dafButler
from lsst.daf.butler import Butler

import lsst.geom as geom
from lsst.geom import SpherePoint, degrees
import lsst.afw.display as afwDisplay

from lsst.skymap import PatchInfo, Index2D

In [ ]:
# For Angle conversion
from astropy.coordinates import Angle
import astropy.units as u

In [ ]:
#%matplotlib widget

In [ ]:
all_bands = ["u", "g", "r", "i", "z", "y"]
all_bands_colors = ["blue", "green", "red", "orange", "yellow", "purple"]

In [ ]:
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.labelsize"] = "xx-large"
plt.rcParams["axes.titlesize"] = "xx-large"
plt.rcParams["xtick.labelsize"] = "xx-large"
plt.rcParams["ytick.labelsize"] = "xx-large"


In [ ]:
def nJy_to_ab_mag(f_njy):
    """Convert scalar or array flux in nJy to AB magnitude."""
    f_njy = np.asarray(f_njy)
    mag = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = f_njy > 0
    mag[mask] = -2.5 * np.log10(f_njy[mask]) + 31.4
    return mag


def nJy_err_to_ab_err(f_njy, f_err):
    """Propagate flux error to magnitude error."""
    f_njy = np.asarray(f_njy)
    f_err = np.asarray(f_err)
    mag_err = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = (f_njy > 0) & (f_err > 0)
    mag_err[mask] = (2.5 / np.log(10)) * (f_err[mask] / f_njy[mask])
    return mag_err


In [ ]:
def ab_mag_to_nJy(mag_ab):
    """Convert AB magnitude to flux in nanojanskys."""
    return 10 ** ((31.4 - mag_ab) / 2.5)


In [ ]:
def plot_tract_patches_ra_dec(butler, skymap, tract_id, df_points=None):
    # Ouvre le Butler
    
    tract_info = skymap[tract_id]
    wcs = tract_info.getWcs()
    num_patches_x, num_patches_y = tract_info.getNumPatches()

    fig, ax = plt.subplots(figsize=(12, 10))

    for y in range(num_patches_y):
        for x in range(num_patches_x):
            patch_info = tract_info.getPatchInfo((x, y))
            patch_seqnum = patch_info.getSequentialIndex()
            patch_bbox = patch_info.getOuterBBox()

            # Coins du patch
            corners = [
                (patch_bbox.getMinX(), patch_bbox.getMinY()),
                (patch_bbox.getMaxX(), patch_bbox.getMinY()),
                (patch_bbox.getMaxX(), patch_bbox.getMaxY()),
                (patch_bbox.getMinX(), patch_bbox.getMaxY()),
            ]
            sky_coords = [wcs.pixelToSky(xp, yp) for xp, yp in corners]
            ra = [coord.getRa().asDegrees() for coord in sky_coords]
            dec = [coord.getDec().asDegrees() for coord in sky_coords]
            ra.append(ra[0])
            dec.append(dec[0])
            ax.plot(ra, dec, color="blue", linewidth=0.5)

            # Centre du patch (en pixels)
            center_pixel_x = 0.5 * (patch_bbox.getMinX() + patch_bbox.getMaxX())
            center_pixel_y = 0.5 * (patch_bbox.getMinY() + patch_bbox.getMaxY())
            center_sky = wcs.pixelToSky(center_pixel_x, center_pixel_y)
            center_ra = center_sky.getRa().asDegrees()
            center_dec = center_sky.getDec().asDegrees()

            # Numéro de patch (x, y)
            #ax.text(center_ra, center_dec, f"{x},{y}", fontsize=6, ha='center', va='center', color='black')
            ax.text(center_ra, center_dec, f"{patch_seqnum}", fontsize=10, ha='center', va='center', color='darkgreen')

    # Ajout des points (optionnel)
    if df_points is not None:
        df = df_points[df_points["tract"] == tract_id]
        ax.scatter(df["ra"], df["dec"], color="red", marker="o", s=30, label="Sources", zorder=5)

        # Optionnel : noms des sources
        for _, row in df.iterrows():
            ax.text(row["ra"], row["dec"], row["field_name"], fontsize=10, fontweight="bold" ,ha='right', va='bottom', color='darkred')

    ax.set_xlabel("RA [deg]")
    ax.set_ylabel("Dec [deg]")
    ax.set_title(f"Patches in Tract {tract_id}")
    ax.invert_xaxis()
    ax.set_aspect('equal', adjustable='datalim')
    ax.grid(True)
    ax.legend()
    plt.tight_layout()
    plt.show()


## Config

In [ ]:
# The output repo is tagged with the Jira ticket number "DM-40356":
repo = "/repo/main"
# collection = 'LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666' # work
# collection = 'LSSTComCam/runs/DRP/DP1/w_2025_06/DM-48810' # work
collection = "LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"  # work


# bad : crash collection = 'LSSTComCam/runs/DRP/DP1/w_2025_08/DM-49029'

# bad : collection = "LSSTComCam/runs/DRP/20241101_20241211/w_2024_51/DM-48233"

# not working perhaps because I am using w_2025_10 version
# bad : no ccd visit collection = "LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864"
# bad : no ccd visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_15/DM-50050'
# bad : no cce visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864'
# bad : no cce visit collection collection = 'LSSTComCam/runs/DRP/DP1/w_2025_13/DM-49751'
instrument = "LSSTComCam"
skymapName = "lsst_cells_v1"
where_clause = "instrument = '" + instrument + "'"
collectionStr = collection.replace("/", "_")
BANDSEL = "r"  # Most fields were observed in red filter

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
skymap = butler.get("skyMap", skymap=skymapName, collections=collection)

In [ ]:
camera = butler.get("camera", collections=collection, instrument=instrument)

## 1) Build the table of AGN from the Vizier Catalog

In [ ]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:
ECDFS_RA = 53.1
ECDFS_DEC = -27.8
ECDFS_COORD = SpherePoint(ECDFS_RA, ECDFS_DEC , degrees)

In [ ]:
# Coordonnées approximatives du centre du E-CDFS
coord = SkyCoord(ra=ECDFS_RA, dec = ECDFS_DEC, unit=(u.deg, u.deg), frame='icrs')
radius = 30 * u.arcmin

# Exemple avec le catalogue Luo+2017 (7Ms CDF-S)
catalog_id = "J/ApJS/228/2"
result = Vizier(columns=["*"]).query_region(coord, radius=radius, catalog=catalog_id)

In [ ]:
sources_table = result['J/ApJS/228/2/sources']
print(sources_table.colnames)

In [ ]:
# On calcule le log10 de LX
logLX = np.log10(sources_table['LX'])

# Et on l'ajoute comme nouvelle colonne à la table
sources_table['logLX'] = logLX

# Attention au nom exact de la colonne
logLX = sources_table['logLX']
is_agn = logLX > 42

is_agn = sources_table['logLX'] > 42  # AGN si LX > 10^42 erg/s
agn_table = sources_table[is_agn]

In [ ]:
agn_table[['RAJ2000', 'DEJ2000', 'logLX', 'LX', 'zspec']].pprint(max_lines=10)

In [ ]:
galaxy_table = sources_table[sources_table['OType'] == 'Galaxy']

In [ ]:
galaxy_table[['RAJ2000', 'DEJ2000', 'logLX', 'LX', 'zspec']].pprint(max_lines=10)

## Plot info from AGN Vizier catalog

In [ ]:
df_galaxy = galaxy_table.to_pandas()
df_agn = agn_table.to_pandas()

In [ ]:
plt.figure(figsize=(14,5))

plt.subplot(1, 2, 1)
plt.hist(df_agn['zspec'].values, bins=20, color='green', alpha=0.7)
plt.xlabel("Redshift zspec")
plt.ylabel("Nombre de sources")
plt.title("Distribution en redshift")

plt.subplot(1, 2, 2)
plt.hist(df_agn['logLX'].values, bins=20, color='orange', alpha=0.7)
plt.xlabel("log(Lx) [erg/s]")
plt.ylabel("Nombre de sources")
plt.title("Distribution de la luminosité X")

plt.tight_layout()
plt.show()

In [ ]:
all_tracts = []
all_patches = []
all_targetnames = []

count = 0
for key, row in df_agn.iterrows():
    target_ra = row["RAJ2000"]
    target_dec = row["DEJ2000"]
    target_name = f"AGN{count}"
    target_point = SpherePoint(target_ra, target_dec, degrees)

    tract_info = skymap.findTract(target_point)
    patch_info = tract_info.findPatch(target_point)
    #bbox = patch_info.getOuterBBox()

    tractNbSel = tract_info.getId()
    patchNbSel = patch_info.getSequentialIndex()

    all_tracts.append(tractNbSel)
    all_patches.append(patchNbSel)
    all_targetnames.append(target_name)
    
    count +=1

In [ ]:
df_agn["tract"] = all_tracts
df_agn["patch"] = all_patches
df_agn['field_name'] = all_targetnames
df_agn.rename(columns={"RAJ2000": "ra", "DEJ2000": "dec"},inplace = True)

In [ ]:
df_agn

In [ ]:
tract_id = 5063

In [ ]:
plot_tract_patches_ra_dec(butler, skymap, tract_id, df_agn)